In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)
print(mnist.train.num_examples)
print(mnist.validation.num_examples)
print(mnist.test.num_examples)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
55000
5000
10000


In [3]:
# parameter and hyperparameter setting
pixel=28
image_size=pixel*pixel
n_class=10
n_hidden1=500
n_hidden2=500
n_latent=20
batch_size=100
training_epochs=15
total_batch=int(mnist.train.num_examples/batch_size)
print(total_batch)

lr=0.001

display_step=1
example_to_show=10

550


In [4]:
x=tf.placeholder('float32',[None,image_size])
keep_prob=tf.placeholder('float32')
z_in=tf.placeholder('float32',[None,n_latent])

In [5]:
def weight_variable(shape):
#     init=tf.Variable(tf.random_normal(shape=shape,dtype='float32'))
    init=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)

def bias_variable(shape):
#     init=tf.Variable(tf.random_normal(shape=shape,dtype='float32'))
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init)

def fc_layer(inputs,n_outputs,activation_fn=tf.nn.relu,keep_prob=1):
    n_inputs=int(inputs.shape[1])
    W_fc=weight_variable([n_inputs,n_outputs])
    b_fc=bias_variable([n_outputs])
    logit_fc=tf.matmul(inputs,W_fc)+b_fc
    _a_fc=activation_fn(logit_fc)
    a_fc=tf.nn.dropout(_a_fc,keep_prob=keep_prob)
    return a_fc,logit_fc

def encoder(inputs,n_hidden1,n_hidden2,n_latent,keep_prob=1):
    fc_layer1,_=fc_layer(inputs,n_hidden1,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    fc_layer2,_=fc_layer(fc_layer1,n_hidden2,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    
    w_encoder=weight_variable([n_hidden2,n_latent*2])
    b_encoder=bias_variable([n_latent*2])
    gaussian_params=tf.matmul(fc_layer2,w_encoder)+b_encoder
    
    mean=gaussian_params[:,:n_latent]
    # to make stddev be positive, add softplus 
    # to make stddev be stable, add a small epsilon
    stddev=1e-6+tf.nn.softplus(gaussian_params[:,n_latent:])
    return mean,stddev

def decoder(z,n_hidden2,n_hidden1,n_outputs,keep_prob=1):
    fc_layer1,_=fc_layer(z,n_hidden2,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    fc_layer2,_=fc_layer(fc_layer1,n_hidden1,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    
    w_decoder=weight_variable([n_hidden1,n_outputs])
    b_decoder=bias_variable([n_outputs])
    outputs=tf.sigmoid(tf.matmul(fc_layer2,w_decoder)+b_decoder)
    
    return outputs

def variational_autoencoder(inputs,n_hidden1,n_hidden2,n_latent,keep_prob=1):
    # encoder
    mu,std=encoder(inputs,n_hidden1,n_hidden2,n_latent,keep_prob)
    
    # sampling by re-parameterization technique
    latent_variable=mu+std*tf.random_normal(tf.shape(mu),mean=0,stddev=1,dtype='float32')
    
    # decoder
    n_outputs=int(inputs.shape[1])
    x_=decoder(latent_variable,n_hidden2,n_hidden1,n_outputs,keep_prob)
    x_=tf.clip_by_value(x_,1e-8,1-1e-8)
    
    # loss
    marginal_likelihood=tf.reduce_sum(inputs*tf.log(x_)+(1-inputs)*tf.log(1-x_),1)
    KL_divergence=0.5*tf.reduce_sum(tf.square(mu)+tf.square(std)-tf.log(1e-7+tf.square(std))-1,1)
    
    marginal_likelihood=tf.reduce_mean(marginal_likelihood)
    KL_divergence=tf.reduce_mean(KL_divergence)
    
    ELBO=marginal_likelihood-KL_divergence
    
    loss=-ELBO
    
    return x_,latent_variable,loss,-marginal_likelihood,KL_divergence

def generator(z,n_hidden2,n_hidden1,n_outputs):
    return decoder(z,n_hidden2,n_hidden1,n_outputs,keep_prob=1)



In [6]:
x_,z,loss,neg_marginal_likelihood,KL_divergence=variational_autoencoder(
    x,n_hidden1,n_hidden2,n_latent,keep_prob=keep_prob)
optimizer=tf.train.AdadeltaOptimizer(learning_rate=lr).minimize(loss)

In [8]:
with tf.Session() as sess:
    init=tf.global_variables_initializer()
    sess.run(init,feed_dict={keep_prob:0.9})

    for epoch in range(training_epochs):
        avg_cost=0.
        for step in range(total_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            
            _,tot_loss,loss_likelihood,loss_divergence=sess.run(
                (optimizer,loss,neg_marginal_likelihood,KL_divergence),
            feed_dict={x:batch_xs,keep_prob:0.9})
            
        print("Epoch: %d: L_tot %03.2f L_likelihood %03.2f L_divergence %03.2f" % (
            epoch,tot_loss,loss_likelihood,loss_divergence))


Epoch: 0: L_tot 665.03 L_likelihood 647.21 L_divergence 17.82
Epoch: 1: L_tot 618.80 L_likelihood 606.23 L_divergence 12.58
Epoch: 2: L_tot 568.99 L_likelihood 560.57 L_divergence 8.42
Epoch: 3: L_tot 523.84 L_likelihood 518.22 L_divergence 5.63
Epoch: 4: L_tot 480.89 L_likelihood 476.75 L_divergence 4.14
Epoch: 5: L_tot 440.40 L_likelihood 437.01 L_divergence 3.39
Epoch: 6: L_tot 401.94 L_likelihood 398.86 L_divergence 3.09
Epoch: 7: L_tot 376.37 L_likelihood 373.27 L_divergence 3.10
Epoch: 8: L_tot 342.77 L_likelihood 339.79 L_divergence 2.98
Epoch: 9: L_tot 314.94 L_likelihood 312.15 L_divergence 2.79
Epoch: 10: L_tot 300.23 L_likelihood 297.45 L_divergence 2.78
Epoch: 11: L_tot 276.58 L_likelihood 274.07 L_divergence 2.51
Epoch: 12: L_tot 271.19 L_likelihood 268.63 L_divergence 2.56
Epoch: 13: L_tot 263.32 L_likelihood 260.82 L_divergence 2.50
Epoch: 14: L_tot 246.17 L_likelihood 243.86 L_divergence 2.31


NameError: name 'avg_cost_list' is not defined

RuntimeError: Attempted to use a closed Session.